In [10]:
import numpy as np
import matplotlib.pyplot as plt
import tarfile
import os
import glob
from collections import defaultdict

In [11]:
def read_lightcurve_file(filename):
    """
    Read the lightcurve file and extract header parameters and data
    """
    data = {}
    
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # Extract header info
    header_params = {}
    data_lines = []
    
    for line in lines:
        if line.startswith('#'):
            # Parse header lines
            if '=' in line:
                key_value = line[1:].strip().split('=')
                if len(key_value) == 2:
                    key = key_value[0].strip()
                    value = key_value[1].strip()
                    header_params[key] = value
        else:
            # Only add non-empty lines that don't start with #
            if line.strip() and not line.strip().startswith('#'):
                data_lines.append(line.strip())
    
    # Convert data to numpy array
    if data_lines:
        try:
            data_array = np.genfromtxt(data_lines)
            
            # Check if we have enough columns
            if data_array.ndim == 1:
                # Single row case
                data_array = data_array.reshape(1, -1)
            
            # Define column names based on your description
            column_names = ['Simulation_time', 'measured_relative_flux', 'measured_relative_flux_error', 
                           'true_relative_flux', 'true_relative_flux_error', 'observatory_code', 
                           'saturation_flag', 'best_single_lens_fit', 'parallax_shift_t', 'parallax_shift_u',
                           'BJD', 'source_x', 'source_y', 'lens1_x', 'lens1_y', 'lens2_x', 'lens2_y']
            
            # Create dictionary with column data
            for i, col_name in enumerate(column_names):
                if i < data_array.shape[1]:
                    data[col_name] = data_array[:, i]
                else:
                    data[col_name] = np.array([])
                    
        except Exception as e:
            print(f"Error parsing data from {filename}: {e}")
            data = None
    else:
        data = None
    
    return header_params, data

In [12]:
def extract_and_process_tar_files():
    """
    Extract all tar.gz files and process the lightcurve files
    """
    # Find all tar.gz files in current directory 
    tar_files = glob.glob('OMPLDG_croin_cassan_*.tar.gz') #glob helps find files whose names have a specific pattern
    
    # If no files found with correct pattern, try the pattern from your error message
    if not tar_files:
        tar_files = glob.glob('OWPLOG_croin_cassan_*.tar.gz')
    
    if not tar_files:
        print("No tar files found! Looking for files in current directory:")
        print("Files in current directory:", os.listdir('.'))
        return {}
    
    print(f"Found {len(tar_files)} tar files to process") #number of tar files found
    all_parameters = {}
    
    for tar_file in tar_files:
        #print(f"Processing {tar_file}...")
        
        # Extract tar file
        with tarfile.open(tar_file, 'r:gz') as tar:
            # Get list of all files in tar
            file_list = tar.getnames()
            #print(f"  Files in archive: {file_list}")
            
            # Extract to a temporary directory
            extract_dir = f"temp_{tar_file.replace('.tar.gz', '')}"
            tar.extractall(extract_dir)
        
        # Find all files in the extracted directory (recursively)
        all_files = []
        for root, dirs, files in os.walk(extract_dir):
            for file in files:
                all_files.append(os.path.join(root, file))
        
        print(f"  Found {len(all_files)} total files in extracted directory")
        
        file_parameters = []
        
        for file_path in all_files:
            # Try to process any file that might be a lightcurve
            try:
                header_params, data = read_lightcurve_file(file_path)
                if data is not None and len(data) > 0 and 'Simulation_time' in data:
                    # Calculate some basic statistics for this lightcurve
                    stats = {
                        'filename': os.path.basename(file_path),
                        'header_params': header_params,
                        'data_stats': {
                            'num_points': len(data['Simulation_time']),
                            'time_range': [np.min(data['Simulation_time']), np.max(data['Simulation_time'])],
                            'mean_measured_flux': np.mean(data['measured_relative_flux']),
                            'std_measured_flux': np.std(data['measured_relative_flux']),
                            'mean_true_flux': np.mean(data['true_relative_flux']),
                            'std_true_flux': np.std(data['true_relative_flux']),
                        },
                        'data': data  # Store the actual data for plotting
                    }
                    file_parameters.append(stats)
                    #print(f"    Successfully processed: {os.path.basename(file_path)} - {len(data['Simulation_time'])} data points")
            except Exception as e:
                # Skip files that can't be processed
                continue
        
        all_parameters[tar_file] = file_parameters
        #print(f"  Successfully processed {len(file_parameters)} lightcurves from {tar_file}")
        
        # Clean up extracted files
        import shutil
        shutil.rmtree(extract_dir)
    
    return all_parameters

In [13]:
def plot_parameter_distributions(all_parameters):
    """
    Create plots to visualize distributions of various parameters
    """
    # Collect data from all files
    all_measured_flux = []
    all_true_flux = []
    all_flux_errors = []
    all_num_points = []
    all_time_ranges = []
    
    # Also collect some example light curves for visualization
    #example_lightcurves = []
    
    for tar_file, file_params_list in all_parameters.items():
        for file_params in file_params_list:
            stats = file_params['data_stats']
            all_measured_flux.append(stats['mean_measured_flux'])
            all_true_flux.append(stats['mean_true_flux'])
            all_flux_errors.append(stats['std_measured_flux'])
            all_num_points.append(stats['num_points'])
            time_range = stats['time_range'][1] - stats['time_range'][0]
            all_time_ranges.append(time_range)
            
            # Store a few example lightcurves for plotting
            #if len(example_lightcurves) < 5:  # Store up to 5 examples
                #example_lightcurves.append(file_params)
    
    # Check if we have any data to plot
    if not all_measured_flux:
        print("No data found to plot!")
        return
    
    # Create subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Light Curve Parameter Distributions', fontsize=16)
    
    # Plot 1: Distribution of mean measured flux
    axes[0, 0].hist(all_measured_flux, bins=20, alpha=0.7, edgecolor='black')
    axes[0, 0].set_xlabel('Mean Measured Relative Flux')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Distribution of Mean Measured Flux')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Distribution of mean true flux
    axes[0, 1].hist(all_true_flux, bins=20, alpha=0.7, edgecolor='black', color='orange')
    axes[0, 1].set_xlabel('Mean True Relative Flux')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Distribution of Mean True Flux')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Distribution of flux errors
    axes[0, 2].hist(all_flux_errors, bins=20, alpha=0.7, edgecolor='black', color='green')
    axes[0, 2].set_xlabel('Standard Deviation of Measured Flux')
    axes[0, 2].set_ylabel('Frequency')
    axes[0, 2].set_title('Distribution of Flux Errors')
    axes[0, 2].grid(True, alpha=0.3)
    
    # Plot 4: Number of data points per lightcurve
    axes[1, 0].hist(all_num_points, bins=20, alpha=0.7, edgecolor='black', color='purple')
    axes[1, 0].set_xlabel('Number of Data Points')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Distribution of Data Points per Lightcurve')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 5: Time range distribution
    axes[1, 1].hist(all_time_ranges, bins=20, alpha=0.7, edgecolor='black', color='red')
    axes[1, 1].set_xlabel('Time Range')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Distribution of Time Ranges')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Plot 6: Example light curves
   # if example_lightcurves:
    #    for i, example in enumerate(example_lightcurves):
         #   data = example['data']
        #    axes[1, 2].plot(data['Simulation_time'], data['measured_relative_flux'], 
                           #'o-', markersize=2, alpha=0.7, label=f'Example {i+1}')
       # axes[1, 2].set_xlabel('Simulation Time')
       # axes[1, 2].set_ylabel('Measured Relative Flux')
        #axes[1, 2].set_title('Example Light Curves')
       # axes[1, 2].legend(fontsize=8)
       # axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('lightcurve_parameter_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Create additional plot: show header parameter information
    plot_header_parameters(all_parameters)


In [14]:
def plot_header_parameters(all_parameters):
    """
    Plot distributions of header parameters if available
    """
    # Collect header parameters
    fs_values = []
    mag_values = []
    
    for tar_file, file_params_list in all_parameters.items():
        for file_params in file_params_list:
            header = file_params['header_params']
            if 'fs' in header:
                try:
                    fs_values.append(float(header['fs']))
                except:
                    pass
            # Look for magnitude parameters
            for key, value in header.items():
                if 'mag' in key.lower():
                    try:
                        mag_values.append(float(value))
                    except:
                        pass
    
    if fs_values or mag_values:
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        fig.suptitle('Header Parameter Distributions', fontsize=16)
        
        if fs_values:
            axes[0].hist(fs_values, bins=20, alpha=0.7, edgecolor='black', color='blue')
            axes[0].set_xlabel('fs values')
            axes[0].set_ylabel('Frequency')
            axes[0].set_title('Distribution of fs parameter')
            axes[0].grid(True, alpha=0.3)
        
        if mag_values:
            axes[1].hist(mag_values, bins=20, alpha=0.7, edgecolor='black', color='green')
            axes[1].set_xlabel('Magnitude values')
            axes[1].set_ylabel('Frequency')
            axes[1].set_title('Distribution of Magnitude parameters')
            axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('header_parameter_distributions.png', dpi=300, bbox_inches='tight')
        plt.show()

In [15]:
def print_summary_statistics(all_parameters):
    """
    Print summary statistics for the dataset
    """
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    
    total_lightcurves = 0
    all_num_points = []
    
    for tar_file, file_params_list in all_parameters.items():
        num_in_file = len(file_params_list)
        total_lightcurves += num_in_file
        print(f"\n{tar_file}: {num_in_file} lightcurves")
        
        for file_params in file_params_list:
            all_num_points.append(file_params['data_stats']['num_points'])
    
    if total_lightcurves == 0:
        print("\nNo lightcurves found in any files!")
        return
    
    print(f"\nTotal lightcurves across all files: {total_lightcurves}")
    if all_num_points:
        print(f"Average number of data points per lightcurve: {np.mean(all_num_points):.1f}")
        print(f"Median number of data points per lightcurve: {np.median(all_num_points):.1f}")
        print(f"Range of data points: {np.min(all_num_points)} - {np.max(all_num_points)}")
    else:
        print("No data points found!")

In [ ]:
def main():
    """
    Main function to process all lightcurve files and generate plots
    """
    #print("Starting lightcurve data analysis...")
    #print("Looking for tar files in current directory...")
    #print("Files found:", [f for f in os.listdir('.') if f.endswith('.tar.gz')])
    
    # Process all tar files
    all_parameters = extract_and_process_tar_files()
    
    # Print summary statistics
    print_summary_statistics(all_parameters)
    
    # Generate distribution plots if we have data
    if any(len(params) > 0 for params in all_parameters.values()):
        plot_parameter_distributions(all_parameters)
        print("\nAnalysis complete!")
    else:
        print("\nNo lightcurve data found to analyze!")

if __name__ == "__main__": 
    main()

Found 4 tar files to process
  Found 114 total files in extracted directory
  Found 97 total files in extracted directory
  Found 98 total files in extracted directory
